In [2]:
import pulp

def status_check(problem):
    val = problem.status
    if val == 1:
        print("\033[92m Optimal")
    elif val == 0:
        print("\033[91m Not Solved") 
    elif val == -1:
        print("\033[91m Infeasible")
    elif val == -2:
        print("\033[91m Unbounded")
    elif val == -3:
        print("\033[91m Undefined")
    else:
        print("\033[91m Unknown status")

    print("\033[39m")

In [13]:
"""
Setup example LP:

max z = x + 2y
s.t. 2x + y <= 20
     -4x + 5y <= 10
     -x + 2y >= -2
     -x + 5y = 15
     x >= 0
     y >= 0
"""

# initialise model (sense = min/max)
model = pulp.LpProblem(name="example", sense=pulp.LpMaximize)


# initialise decision variables (default bounds -inf, inf)
# (can specify contiuous/discrete)
# linear combinations of variables gives expression object
x = pulp.LpVariable(name="x", lowBound=0)
y = pulp.LpVariable(name="y", lowBound=0)

# add constraints specified via tuple: expression, name
model += (2 * x + y <= 20, "constraint_1")
model += (-4 * x + 5 * y <= 10, "constraint_2")
model += (-x + 2 * y >= -2, "constraint_3")
model += (-x + 5 * y == 15, "constraint_4")

# add objective function
model += x + 2 * y

print(model)

# solve
status = model.solve()

model.objective.value()

example:
MAXIMIZE
1*x + 2*y + 0
SUBJECT TO
constraint_1: 2 x + y <= 20

constraint_2: - 4 x + 5 y <= 10

constraint_3: - x + 2 y >= -2

constraint_4: - x + 5 y = 15

VARIABLES
x Continuous
y Continuous



16.8181817

# Birth Death process

In [2]:
"""
Birth death process LP
- k1 = 1/2
- fix k2 = 1
- add equations from Qp = 0 to bound k0 via min and max LP's
"""

# upper bound on k1
model_1 = pulp.LpProblem(name="upper_bound", sense=pulp.LpMaximize)
# positive rate
k1 = pulp.LpVariable(name="k1", lowBound=0)
# constraints
model_1 += (-0.5 * k1 + 0.25 == 0, "constraint_1")
# objective
model_1 += k1
# solve
status = model_1.solve()
print(f"objective: {model_1.objective.value()}")
for var in model_1.variables():
    print(f"{var.name}: {var.value()}")

# lower bound will be same

# NOTE: simply gives a point, as 1 unknown only needs 1 equation

objective: 0.5
k1: 0.5


In [25]:
"""
Birth Death process: sampled values
- k1 to be found
- fix k2 = 1
BUT
- now only have bounds on values of p0, p1, p2, ..., so also variables

- as an example, choose k1 in range 0.5 <-> 0.6, and bound pn's by max and min possible
- 1 eqn gives good bounds, adding a 2nd gives exact bounds
"""

# upper bound on k1
model_1 = pulp.LpProblem(name="upper_bound", sense=pulp.LpMaximize)
# variables
k1 = pulp.LpVariable(name="k1", lowBound=0)
z10 = pulp.LpVariable(name="z10", lowBound=0)
z21 = pulp.LpVariable(name="z21", lowBound=0)
z11 = pulp.LpVariable(name="z11", lowBound=0)
z22 = pulp.LpVariable(name="z22", lowBound=0)
#z12 = pulp.LpVariable(name="z12", lowBound=0)
#z23 = pulp.LpVariable(name="z23", lowBound=0)
# constraints
model_1 += (-z10 + z21 == 0, "eqn_1")
model_1 += (z10 >= k1 * 0.4, "z10_LB")
model_1 += (z10 <= k1 * 0.5, "z10_UB")
model_1 += (z21 >= 1 * 0.24, "z21_LB")
model_1 += (z21 <= 1 * 0.25, "z21_UB")
model_1 += (z10 - z11 - z21 + z22 == 0, "eqn_2")
model_1 += (z11 >= k1 * 0.24, "z11_LB")
model_1 += (z11 <= k1 * 0.25, "z11_UB")
model_1 += (z22 >= 1 * 0.125, "z22_LB")
model_1 += (z22 <= 1 * 0.144, "z22_UB")
#model_1 += (z11 - z12 - z22 + z23 == 0, "eqn_3")
#model_1 += (z12 >= k1 * 3/32, "z12_LB")
#model_1 += (z12 <= k1 * 5/32, "z12_UB")
#model_1 += (z23 >= 1 * 3/64, "z23_LB")
#model_1 += (z23 <= 1 * 5/64, "z23_UB")
# objective
model_1 += k1
# solve
status = model_1.solve()
print(f"objective: {model_1.objective.value()}")
for var in model_1.variables():
    print(f"{var.name}: {var.value()}")

# lower bound on k1
model_1.sense = pulp.LpMinimize
# solve
status = model_1.solve()
print(f"objective: {model_1.objective.value()}")
for var in model_1.variables():
    print(f"{var.name}: {var.value()}")


"""
Results:
1 equation + bounds: k1 in (0.48, 0.625)
2 equations + bounds: k1 in (0.5, 0.6) [exact bounds]
3 equations + bounds: same
"""

objective: 0.6
k1: 0.6
z10: 0.24
z11: 0.144
z21: 0.24
z22: 0.144
objective: 0.5
k1: 0.5
z10: 0.24
z11: 0.125
z21: 0.24
z22: 0.125


'\nResults:\n1 equation + bounds: k1 in (0.3, 0.8333..)\n2 equations + bounds: same\n3 equations + bounds: same\n'

In [15]:
"""
Second example of sampled value bounds
- k1 to be estimated
- fix k2 = 1

exact stationary dist:
- p0 = 1 - k1/k2
- pn = (k1/k2)^n * (1 - k1/k2)

choose k1 in range [0.1, 0.4]:
- use min/max stat dist possible as bounds

Adding noise:
- add small variation to bounds
- see that bounds also change only slightly
"""

# stationary distribution bounds
p0l, p1l, p2l, p3l = 0.6,0.09,0.009, 0
p0u, p1u, p2u, p3u = 0.9,0.24,0.096, 0

# upper bound on k1
model_1 = pulp.LpProblem(name="upper_bound", sense=pulp.LpMaximize)
# variables
k1 = pulp.LpVariable(name="k1", lowBound=0)
k2 = 1
z10 = pulp.LpVariable(name="z10", lowBound=0)
z21 = pulp.LpVariable(name="z21", lowBound=0)
z11 = pulp.LpVariable(name="z11", lowBound=0)
z22 = pulp.LpVariable(name="z22", lowBound=0)
z12 = pulp.LpVariable(name="z12", lowBound=0)
z23 = pulp.LpVariable(name="z23", lowBound=0)
# constraints
model_1 += (-z10 + z21 == 0, "eqn_1")
model_1 += (z10 >= k1 * p0l, "z10_LB")
model_1 += (z10 <= k1 * p0u, "z10_UB")
model_1 += (z21 >= k2 * p1l, "z21_LB")
model_1 += (z21 <= k2 * p1u, "z21_UB")
#model_1 += (z10 - z11 - z21 + z22 == 0, "eqn_2")
#model_1 += (z11 >= k1 * p1l, "z11_LB")
#model_1 += (z11 <= k1 * p1u, "z11_UB")
#model_1 += (z22 >= 1 * p2l, "z22_LB")
#model_1 += (z22 <= 1 * p2u, "z22_UB")
#model_1 += (z11 - z12 - z22 + z23 == 0, "eqn_3")
#model_1 += (z12 >= k1 * 3/32, "z12_LB")
#model_1 += (z12 <= k1 * 5/32, "z12_UB")
#model_1 += (z23 >= 1 * 3/64, "z23_LB")
#model_1 += (z23 <= 1 * 5/64, "z23_UB")
# objective
model_1 += k1
# solve
status = model_1.solve()
status_check(model_1)
print(f"objective: {model_1.objective.value()}")
for var in model_1.variables():
    print(f"{var.name}: {var.value()}")

# lower bound on k1
model_1.sense = pulp.LpMinimize
# solve
status = model_1.solve()
status_check(model_1)
print(f"objective: {model_1.objective.value()}")
for var in model_1.variables():
    print(f"{var.name}: {var.value()}")


"""
Results:
1 equation + bounds: k1 in (0.1, 0.4) [exact bounds]

Attempt to perturbe:
small changes to bounds of k1
finds exact bounds given noisy bounds, are close to original
"""

 Optimal

objective: 0.4
k1: 0.4
z10: 0.24
z21: 0.24
 Optimal

objective: 0.1
k1: 0.1
z10: 0.09
z21: 0.09


'\nResults:\n1 equation + bounds: k1 in (0.1, 0.4) [exact bounds]\n\nAttempt to perturbe:\nsmall changes to bounds of k1\nfinds exact bounds given noisy bounds, are close to original\n'

# Sclogl's Model

In [19]:
"""
Schlog's model: fixed values
- choose values of k1, k2, k3, k4
- use fixed values of stat dist p0, p1, p2, ...
- obtain stat dist via simulation

"""
# estimate of stat dist for params = (1,1,0.5,1)
# p0, p1, p2, ... = 0.6, 0.3, 0.075, 0.0125, 0.000223, ...
# stationary distribution bounds
p0l, p1l, p2l, p3l = 0.6,0.3,0.075,0.0125
p0u, p1u, p2u, p3u = 0.6,0.3,0.075,0.0125

# upper bound on k1
model_2 = pulp.LpProblem(name="upper_bound", sense=pulp.LpMaximize)
# variables
# fix k2, k4 as 1 intially
k1 = pulp.LpVariable(name="k1", lowBound=0)
#k2 = pulp.LpVariable(name="k2", lowBound=0)
k3 = pulp.LpVariable(name="k3", lowBound=0)
#k4 = pulp.LpVariable(name="k4", lowBound=0)
k2 = 1
k4 = 1
z12 = pulp.LpVariable(name="z12", lowBound=0)
z23 = pulp.LpVariable(name="z23", lowBound=0)
z30 = pulp.LpVariable(name="z30", lowBound=0)
z31 = pulp.LpVariable(name="z31", lowBound=0)
z32 = pulp.LpVariable(name="z32", lowBound=0)
z41 = pulp.LpVariable(name="z41", lowBound=0)
z42 = pulp.LpVariable(name="z42", lowBound=0)
z43 = pulp.LpVariable(name="z43", lowBound=0)
# constraints
model_2 += (-z30 + z41 == 0, "eqn_1")
model_2 += (z30 >= k3 * p0l, "z30_LB")
model_2 += (z30 <= k3 * p0u, "z30_UB")
model_2 += (z41 >= k4 * p1l, "z41_LB")
model_2 += (z41 <= k4 * p1u, "z41_UB")
model_2 += (z30 - z31 - z41 + 2 * z42 == 0, "eqn_2")
model_2 += (z31 >= k3 * p1l, "z31_LB")
model_2 += (z31 <= k3 * p1u, "z31_UB")
model_2 += (z42 >= k4 * p2l, "z42_LB")
model_2 += (z42 <= k4 * p2u, "z42_UB")
model_2 += (-2 * z12 + 6 * z23 + z31 - z32 - 2 * z42 + 3 * z43 == 0, "eqn_3")
model_2 += (z12 >= k1 * p2l, "z12_LB")
model_2 += (z12 <= k1 * p2u, "z12_UB")
model_2 += (z23 >= k2 * p3l, "z23_LB")
model_2 += (z23 <= k2 * p3u, "z23_UB")
model_2 += (z32 >= k3 * p2l, "z32_LB")
model_2 += (z32 <= k3 * p2u, "z32_UB")
model_2 += (z43 >= k4 * p3l, "z43_LB")
model_2 += (z43 <= k4 * p3u, "z43_UB")
# objective
model_2 += k1
# solve
status = model_2.solve()
status_check(model_2)
print(f"objective: {model_2.objective.value()}")
for var in model_2.variables():
    print(f"{var.name}: {var.value()}")

# lower bound on k1
model_2.sense = pulp.LpMinimize
# solve
status = model_2.solve()
status_check(model_2)
print(f"objective: {model_2.objective.value()}")
for var in model_2.variables():
    print(f"{var.name}: {var.value()}")


"""
Results:
fixing k1 = 1, k4 = 1:
1 eqn: exact value of k3
3 eqn: exact value of k1
"""

 Optimal

objective: 0.5
k1: 0.5
k3: 0.5
z12: 0.0375
z23: 0.0125
z30: 0.3
z31: 0.15
z32: 0.0375
z41: 0.3
z42: 0.075
z43: 0.0125
 Optimal

objective: 0.5
k1: 0.5
k3: 0.5
z12: 0.0375
z23: 0.0125
z30: 0.3
z31: 0.15
z32: 0.0375
z41: 0.3
z42: 0.075
z43: 0.0125


'\nResults:\nfixing k1 = 1, k4 = 1:\n1 eqn: exact value of k3\n3 eqn: exact value of k1\n'

In [25]:
# Using bounds

# estimate of stat dist for params = (1,1,0.5,1)
# p0, p1, p2, ... = 0.6, 0.3, 0.075, 0.020833, 0.004836, ...
# estimate of stat dist for params = (1,1,0.6,1)
# p0, p1, p2, ... = 0.545, 0.2725, 0.1, 0.003, 0.001
# stationary distribution bounds
p0l, p1l, p2l, p3l, p4l = 0.545,0.2725,0.068125,0.018,0
p0u, p1u, p2u, p3u, p4u = 0.6,0.3,0.075,0.021,0.005

# upper bound on k1
model_2 = pulp.LpProblem(name="upper_bound", sense=pulp.LpMaximize)
# variables
# fix k2, k4 as 1 intially
k1 = pulp.LpVariable(name="k1", lowBound=0)
#k2 = pulp.LpVariable(name="k2", lowBound=0)
k3 = pulp.LpVariable(name="k3", lowBound=0)
#k4 = pulp.LpVariable(name="k4", lowBound=0)
k2 = 1
k4 = 1
z12 = pulp.LpVariable(name="z12", lowBound=0)
z13 = pulp.LpVariable(name="z13", lowBound=0)
z23 = pulp.LpVariable(name="z23", lowBound=0)
z24 = pulp.LpVariable(name="z24", lowBound=0)
z30 = pulp.LpVariable(name="z30", lowBound=0)
z31 = pulp.LpVariable(name="z31", lowBound=0)
z32 = pulp.LpVariable(name="z32", lowBound=0)
z33 = pulp.LpVariable(name="z33", lowBound=0)
z41 = pulp.LpVariable(name="z41", lowBound=0)
z42 = pulp.LpVariable(name="z42", lowBound=0)
z43 = pulp.LpVariable(name="z43", lowBound=0)
z44 = pulp.LpVariable(name="z44", lowBound=0)
# constraints
model_2 += (-z30 + z41 == 0, "eqn_1")
model_2 += (z30 >= k3 * p0l, "z30_LB")
model_2 += (z30 <= k3 * p0u, "z30_UB")
model_2 += (z41 >= k4 * p1l, "z41_LB")
model_2 += (z41 <= k4 * p1u, "z41_UB")
model_2 += (z30 - z31 - z41 + 2 * z42 == 0, "eqn_2")
model_2 += (z31 >= k3 * p1l, "z31_LB")
model_2 += (z31 <= k3 * p1u, "z31_UB")
model_2 += (z42 >= k4 * p2l, "z42_LB")
model_2 += (z42 <= k4 * p2u, "z42_UB")
model_2 += (-2 * z12 + 6 * z23 + z31 - z32 - 2 * z42 + 3 * z43 == 0, "eqn_3")
model_2 += (z12 >= k1 * p2l, "z12_LB")
model_2 += (z12 <= k1 * p2u, "z12_UB")
model_2 += (z23 >= k2 * p3l, "z23_LB")
model_2 += (z23 <= k2 * p3u, "z23_UB")
model_2 += (z32 >= k3 * p2l, "z32_LB")
model_2 += (z32 <= k3 * p2u, "z32_UB")
model_2 += (z43 >= k4 * p3l, "z43_LB")
model_2 += (z43 <= k4 * p3u, "z43_UB")
model_2 += (2 * z12 - 6 * z13 - 6 * z23 + 24 * z24 + z32 - z33 - 3 * z43 + 4 * z44 == 0, "eqn_4")
model_2 += (z13 >= k1 * p3l, "z13_LB")
model_2 += (z13 <= k1 * p3u, "z13_UB")
model_2 += (z24 >= k2 * p4l, "z24_LB")
model_2 += (z24 <= k2 * p4u, "z24_UB")
model_2 += (z33 >= k3 * p3l, "z33_LB")
model_2 += (z33 <= k3 * p3u, "z33_UB")
model_2 += (z44 >= k4 * p4l, "z44_LB")
model_2 += (z44 <= k4 * p4u, "z44_UB")
# objective
model_2 += k3
# solve
status = model_2.solve()
status_check(model_2)
print(f"objective: {model_2.objective.value()}")
for var in model_2.variables():
    print(f"{var.name}: {var.value()}")

# lower bound on kr
model_2.sense = pulp.LpMinimize
# solve
status = model_2.solve()
status_check(model_2)
print(f"objective: {model_2.objective.value()}")
for var in model_2.variables():
    print(f"{var.name}: {var.value()}")

"""
Results:
1 eqn: k3 [0.45, 0.55] (rough approx) (no k1 info)
2 eqn: unchanged
3 eqn:  k3 [0.45, 0.55] k1 [0.8,1.16]
4 eqn: unchanged

bounds good, but slightly innaccurate, likely due to errors in estimate of
stationary dist bounds (i.e. cut off k3 = 0.6, allow wider range of k1)
"""

 Optimal

objective: 0.55045872
k1: 0.83
k3: 0.55045872
z12: 0.06225
z13: 0.01494
z23: 0.018
z24: 0.004147844
z30: 0.3
z31: 0.15
z32: 0.0375
z33: 0.0099082569
z41: 0.3
z42: 0.075
z43: 0.018
z44: 0.0
 Optimal

objective: 0.45416667
k1: 0.87373264
k3: 0.45416667
z12: 0.065529948
z13: 0.015727188
z23: 0.018
z24: 0.0042724219
z30: 0.2725
z31: 0.13625
z32: 0.030940104
z33: 0.008175
z41: 0.2725
z42: 0.068125
z43: 0.018
z44: 0.0


'\nResults:\n1 eqn: k3 [0.45, 0.55] (rough approx) (no k1 info)\n2 eqn: unchanged\n3 eqn:  k3 [0.45, 0.55] k1 [0.8,1.16]\n4 eqn: unchanged\n\nrough bounds, but likely due to wide range of estimated probabilities\nso minimal possible bounds on k3 likely larger than [0,5, 0.6]\n'

# CVXPY

Allows using matrices and vectors for constraints and objective

In [1]:
import cvxpy as cv
import numpy as np

In [4]:
"""
Trial of Birth Death LP via matrices
"""

# number of equations used (rows of Q)
N = 1

# NOTE: define one extra row and cut off, since square matrix would leave out
# upper diagonal elements on Nth row

# create Qr matrices
Q1 = (np.diag(np.ones(N),-1) + np.diag(-np.ones(N + 1),0))[:-1, :]
Q2 = (np.diag(np.ones(N),1) + np.diag(-np.ones(N + 1),0))[:-1, :]
Q2[0,0] = 0

# define data
p = np.array([(0.5) ** i for i in range(1,N + 2)])

a1 = Q1 @ p
a2 = Q2 @ p

# Construct the problem.
k1 = cv.Variable(1)
k2 = 1
objective = cv.Minimize(k1)
constraints = [a1 * k1 + a2 == 0, k1 >= 0]
prob = cv.Problem(objective, constraints)   

# Print result.
result = prob.solve()
print("\nThe optimal value is", prob.value)
print("A solution k1 is")
print(k1.value)


The optimal value is 0.49999999999999367
A solution k1 is
[0.5]


c:\Users\willi\principles_of_programming\PoP_venv\lib\site-packages\cvxpy\expressions\expression.py:612: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 1 times so far.

  warnings.warn(msg, UserWarning)


Now use bounds

In [148]:
# number of equations used (rows of Q)
N = 2

# NOTE: define one extra row and cut off, since square matrix would leave out
# upper diagonal elements on Nth row

# create Qr matrices
Q1 = (np.diag(np.ones(N),-1) + np.diag(-np.ones(N + 1),0))[:-1, :]
Q2 = (np.diag(np.ones(N),1) + np.diag(-np.ones(N + 1),0))[:-1, :]
Q2[0,0] = 0

# define bounds (found from k1 ranging in [0.5, 0.6])
pl = np.array([0.4, 0.24, 0.125, 0.0625])[:N + 1]
pu = np.array([0.5, 0.25, 0.144, 0.0864])[:N + 1]

# NOTE: truncate to N + 1, as tridiagonal, so Nth equation has N+1 p's

# Construct the problem.
k1 = cv.Variable(1)
k2 = 1
z1 = cv.Variable(N + 1)
z2 = cv.Variable(N + 1)
objective_max = cv.Maximize(k1)
objective_min = cv.Minimize(k1)
constraints = [Q1 @ z1 + Q2 @ z2 == 0,
               k1 >= 0,
               z1 >= k1 * pl,
               z1 <= k1 * pu,
               z2 >= k2 * pl,
               z2 <= k2 * pu]
prob_max = cv.Problem(objective_max, constraints)
prob_min = cv.Problem(objective_min, constraints)   

# Print result.
result_max = prob_max.solve()
print("\nThe upper bound is", prob_max.value)
print("A solution k1 is")
print(k1.value)
result_min = prob_min.solve()
print("\nThe lower bound is", prob_min.value)
print("A solution k1 is")
print(k1.value)




The upper bound is 0.6000000004948876
A solution k1 is
[0.6]

The lower bound is 0.4999999999484507
A solution k1 is
[0.5]


# Schlogl: cvxpy

In [7]:
# number of equations used (rows of Q)
N = 3

# NOTE: define one extra row and cut off, since square matrix would leave out
# upper diagonal elements on Nth row

# create Qr matrices
Q1 = (np.diag([-x*(x-1) for x in range(0,N+1)],0) + np.diag([x*(x-1) for x in range(0,N)],-1))[:-1, :]
Q2 = (np.diag([-x*(x-1)*(x-2) for x in range(0,N+1)],0) + np.diag([x*(x-1)*(x-2) for x in range(1,N+1)],1))[:-1, :]
Q3 = (np.diag([-1 for x in range(0,N+1)],0) + np.diag([1 for x in range(0,N)],-1))[:-1, :]
Q4 = (np.diag([-x for x in range(0,N+1)],0) + np.diag([x for x in range(1,N+1)],1))[:-1, :]

# define bounds
pl = np.array([0.545,0.2725,0.068125,0.018,0,0,0,0])[:N + 1]
pu = np.array([0.6,0.3,0.075,0.021,0.005,0,0,0])[:N + 1]

# NOTE: truncate to N + 1, as tridiagonal, so Nth equation has N+1 p's

# Construct the problem.
k1 = cv.Variable(1)
#k2 = cv.Variable(1)
k2 = 1
k3 = cv.Variable(1)
#k4 = cv.Variable(1)
k4 = 1
z1 = cv.Variable(N + 1)
z2 = cv.Variable(N + 1)
z3 = cv.Variable(N + 1)
z4 = cv.Variable(N + 1)
objective_max = cv.Maximize(k1)
objective_min = cv.Minimize(k1)
constraints = [Q1 @ z1 + Q2 @ z2 + Q3 @ z3 + Q4 @ z4 == 0,
               k1 >= 0, k2 >= 0, k3 >= 0, k4 >= 0,
               k1 * pl <= z1, z1 <= k1 * pu,
               k2 * pl <= z2, z2 <= k2 * pu,
               k3 * pl <= z3, z3 <= k3 * pu,
               k4 * pl <= z4, z4 <= k4 * pu,
               ]
prob_max = cv.Problem(objective_max, constraints)
prob_min = cv.Problem(objective_min, constraints)   

# Print result.
result_max = prob_max.solve()
print("\nThe upper bound is", prob_max.value)
print(f"k1: {k1.value} \nk3: {k3.value}")
result_min = prob_min.solve()
print("\nThe lower bound is", prob_min.value)
print(f"k1: {k1.value} \nk3: {k3.value}")


The upper bound is 1.1600726302985909
k1: [1.16007263] 
k3: [0.45416667]

The lower bound is 0.8047706420078607
k1: [0.80477064] 
k3: [0.55045872]
